In [ ]:
import sys,os

RES_PATH = 'meshes' 

if not os.path.exists(RES_PATH):
    print( 'cannot find meshes  please update RES_PATH')
    exit(1)
else:
    pass

import igl
import scipy as sp
import numpy as np
import meshplot as mp
import trimesh
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from os import listdir
from os.path import isfile, join
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix

# 1. Uniform Laplace

## Mean Curvature

In [ ]:
def compute_lap_uniform(mesh):
    num_vertices = mesh.vertices.shape[0]
    # create empty matrix to store laplace operator
    L = np.zeros([num_vertices, num_vertices])
    # loop through all vertices
    for i in range(num_vertices):
        # get the neighbors of current vertex
        neighbors = mm.vertex_neighbors[i]
        # count the number of neighbors
        num_neigbors = len(neighbors)
        # set diagonal
        L[i, i] = num_neigbors
        # set neighbors to -1
        for neighbor in neighbors:
            L[i, neighbor] = -1
        # divide by the number of neighbors
        L[i, :] = L[i, :] / num_neigbors
    L = csc_matrix(L)
    return L

In [ ]:
def compute_mean_curvature(L, vertices):
    # compute mean curvature
    mean_curvature = np.linalg.norm(L @ vertices, axis=1) / 2
    # normalize to a color map
    norm = Normalize(vmin=np.min(mean_curvature), vmax=np.max(mean_curvature))
    cmap = cm.get_cmap('viridis')
    colors = cmap(norm(mean_curvature))[:, :3]
    
    return mean_curvature, colors

In [ ]:
def shade_by_color(mesh, color):
    shading = {"width": 300, "height": 300}
    pt = mp.plot(mesh.vertices, mesh.faces, shading=shading, c=color)

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'bumpy-cube-small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

L = compute_lap_uniform(mm)
mean_curvature, colors = compute_mean_curvature(L, mm.vertices)
shade_by_color(mm, colors)

## Gaussian Curvature 

In [ ]:
def compute_angle_area(vertices, vertex, neighbors):
    # find the angle at the current vertex
    # by taking the dot product between edges (as vectors)
    vec1 = vertices[vertex] - mm.vertices[neighbors[0]]
    vec2 = vertices[vertex] - mm.vertices[neighbors[1]]
    vec1_normalized = vec1 / np.linalg.norm(vec1)
    vec2_normalized = vec2 / np.linalg.norm(vec2)
    angle = np.arccos(np.dot(vec1_normalized, vec2_normalized))

    # compute the area of the triangle
    # half cross product formula
    area = np.linalg.norm(np.cross(vec1, vec2)) / 2
    return angle, area

In [ ]:
def compute_gauss_curvature(mm):
    acc_angles = np.zeros(mm.vertices.shape[0])
    acc_areas = np.zeros(mm.vertices.shape[0])

    # for each vertex
    for i in range(mm.vertices.shape[0]):
        # get its connected faces
        vertex_faces = mm.vertex_faces[i]
        # for each connect face
        for face in vertex_faces:
            # if not an empty entry
            if face != -1:
                face_vertices = mm.faces[face]
                dup = list(face_vertices).copy()
                dup.remove(i)
                # compute angle and area of current face at current vertex
                angle, area = compute_angle_area(mm.vertices, i, dup)
                
                # using built in fns instead
                #idx = np.argmin(np.abs(face_vertices - i))
                #acc_angles[i] += mm.face_angles[face, idx]
                #acc_areas[i] += mm.area_faces[face]

                acc_angles[i] += angle
                acc_areas[i] += area

    # barycentric cell, area = 1/3 triangle areas
    A = acc_areas * (1/3)
    # compute gaussian curvature
    gaussian_curvature = (2 * np.pi - acc_angles) / A

    # normalize to a color map
    norm = Normalize(vmin=np.min(gaussian_curvature), vmax=np.max(gaussian_curvature))
    cmap = cm.get_cmap('viridis')
    colors = cmap(norm(gaussian_curvature))[:, :3]

    return gaussian_curvature, colors

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'bumpy-cube-small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh) 

gauss_curvature, colors = compute_gauss_curvature(mm)
shade_by_color(mm, colors)

## 3. Non-uniform (Discrete Laplace-Beltrami)

In [ ]:
def compute_lap_cot(mm):
    num_vertices = mm.vertices.shape[0]
    # create empty matrix to store laplace operator
    C = np.zeros([num_vertices, num_vertices])
    C = lil_matrix(C)
    M = np.zeros(num_vertices)

    # loop through all vertices
    for i in range(num_vertices):
        # get the neighbors of current vertex
        neighbors = mm.vertex_neighbors[i]
        # for each neighbor
        for neighbor in neighbors:
            angles = []
            # find faces that contains the edge between vertex & current neighbor
            # for each face
            for face in mm.vertex_faces[i]:
                # if not an empty entry
                if face != -1:
                    face_vertices = mm.faces[face]
                    # if face share this edge
                    if neighbor in face_vertices:
                        # get angle of the third vertex
                        # first find the index of the third vertex
                        dup = list(face_vertices).copy()
                        dummy_idx = [0, 1, 2]
                        # removes the index of the current neighbor and vertex
                        # the index remaining is the vertex we are looking for
                        dummy_idx.remove(dup.index(neighbor))
                        dummy_idx.remove(dup.index(i))
                        assert len(dummy_idx) == 1
                        # get the angle
                        angles.append(mm.face_angles[face, dummy_idx[0]])
                        
            # get sum of cotans 
            # remove 0s if there are any
            angles = [value for value in angles if value != 0]
            angles = np.array(angles)
            cot_angles_sum = np.sum(1 / np.tan(angles)) / 2

            # construct C matrix
            C[i, neighbor] = cot_angles_sum
            C[i, i] -= cot_angles_sum

        # for every connected face
        for face in mm.vertex_faces[i]:
            # if not an empty entry
            if face != -1:
                # sum up triangle areas connected to the current vertex
                M[i] += mm.area_faces[face]
    
    # compute laplace beltrami operator
    M = M / 3
    M_inv = sp.sparse.spdiags([1 / M], np.array([0]))
    C = C.tocsc()
    L = M_inv @ C

    return L, M, C

In [ ]:
# load mesh
mesh = os.path.join(RES_PATH,'bumpy-cube-small.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)

L, M, C = compute_lap_cot(mm)
mean_curvature, colors = compute_mean_curvature(L, mm.vertices)

shade_by_color(mm, colors)

# 4. Modal Analysis

In [ ]:
def modal_analysis(mesh, k=200):
    _, M, C = compute_lap_cot(mesh)
    # https://www.cs.jhu.edu/~misha/ReadingSeminar/Papers/Vallet08.pdf
    # equation (4)
    hodge_star_inv = sp.sparse.spdiags([M ** (-0.5)], np.array([0]))
    symmetric_lap = hodge_star_inv @ C @ hodge_star_inv
    # compute eigne vectors, find k smallest 
    vals, vecs = sp.sparse.linalg.eigs(symmetric_lap, k, which='SM')
    # section 2.4
    # map into canonical basis
    basis = hodge_star_inv @ vecs    
    basis = basis.real

    return basis, M

In [ ]:
def reconstruction(mesh, k, basis, M):
    new_vertices = np.zeros(mesh.vertices.shape)
    for vec in basis[:,:k].T:
        # equation (7) and equation (6)
        new_vertices[:, 0] += np.sum(mesh.vertices[:, 0] * M * vec) * vec
        new_vertices[:, 1] += np.sum(mesh.vertices[:, 1] * M * vec) * vec
        new_vertices[:, 2] += np.sum(mesh.vertices[:, 2] * M * vec) * vec

    return new_vertices

In [ ]:
mesh = os.path.join(RES_PATH,'armadillo.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)

shading = {"width": 300, "height": 300}
# compute the basis vectors
basis, M = modal_analysis(mm)

In [ ]:
# perform reconstruction
new_vertices = reconstruction(mm, 100, basis, M)
mp.plot(new_vertices, mm.faces, shading=shading)

# 5. Explicit Laplacian Mesh Smoothing

In [ ]:
mesh = os.path.join(RES_PATH,'smoothing/fandisk_ns.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)

In [ ]:
def explicit_smooth(mesh, itrs, lammy):
    for i in range(itrs):
        L, _, _ = compute_lap_cot(mesh)
        new_vertices = mesh.vertices + lammy * L @ mesh.vertices
        mesh.vertices = new_vertices

In [ ]:
mm_dup = mm.copy()
explicit_smooth(mm_dup, 4, 1e-3)

In [ ]:
shading = {"width": 300, "height": 300}
mp.plot(mm_dup.vertices, mm_dup.faces, shading=shading)

# 6. Implicit Laplacian Mesh Smoothing

In [ ]:
mesh = os.path.join(RES_PATH,'smoothing/fandisk_ns.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)

In [ ]:
def implicit_smooth(mesh, itrs, lammy):
    for i in range(itrs):
        _, M, C = compute_lap_cot(mesh)
        M = sp.sparse.spdiags([M], np.array([0]))
        # conjugate gradients takes in (N, 1) vector for b so processing one dimension at a time
        # solving (M - lambda * C) P_(t-1) = M P_(t)
        new_vertices_x, exit_code = sp.sparse.linalg.cg(M - lammy * C, M @ mesh.vertices[:,0])
        assert exit_code == 0
        new_vertices_y, exit_code = sp.sparse.linalg.cg(M - lammy * C, M @ mesh.vertices[:,1])
        assert exit_code == 0
        new_vertices_z, exit_code = sp.sparse.linalg.cg(M - lammy * C, M @ mesh.vertices[:,2])
        assert exit_code == 0
        mesh.vertices = np.stack([new_vertices_x, new_vertices_y, new_vertices_z]).T

In [ ]:
mm_dup = mm.copy()
implicit_smooth(mm_dup, 2, 1e-1)

In [ ]:
shading = {"width": 300, "height": 300}
mp.plot(mm_dup.vertices, mm_dup.faces, shading=shading)

# 7. Denoising Performance Evaluation

In [ ]:
def add_noise(k, mesh):
    # make a duplicate mesh
    dup = mesh.copy()
    vertices = mesh.vertices
    num_vertices = vertices.shape[0]

    # compute sigma based on the size of the bounding box in each dimension
    sigma_x = np.abs(np.min(vertices[:, 0]) - np.max(vertices[:, 0])) * k
    sigma_y = np.abs(np.min(vertices[:, 1]) - np.max(vertices[:, 1])) * k
    sigma_z = np.abs(np.min(vertices[:, 2]) - np.max(vertices[:, 2])) * k

    # generate 0 mean gaussian noise with the sigma
    x_noise = np.random.normal(0, sigma_x, num_vertices)
    y_noise = np.random.normal(0, sigma_y, num_vertices)
    z_noise = np.random.normal(0, sigma_z, num_vertices)
    # add the generated noise to vertex positions
    noise = np.stack((x_noise, y_noise, z_noise)).reshape(num_vertices, 3)
    
    # add noise
    noisy_vertices = vertices + noise
    dup.vertices = noisy_vertices

    # return new noisy mesh
    return dup

In [ ]:
mesh = os.path.join(RES_PATH,'bunny.obj')
assert os.path.exists(mesh), 'cannot found:'+ mesh 
mm = trimesh.load(mesh)
noisy = add_noise(0.005, mm)

In [ ]:
shading = {"width": 300, "height": 300}
mp.plot(noisy.vertices, noisy.faces, shading=shading)

In [ ]:
mm_dup = noisy.copy()
implicit_smooth(mm_dup, 4, 3e-6)

In [ ]:
shading = {"width": 300, "height": 300}
mp.plot(mm_dup.vertices, mm_dup.faces, shading=shading)